# Imports

In [1]:
from collections import defaultdict
from itertools import product
import numpy as np
import pandas as pd

# Data

In [2]:
bestand = '.\data\dag12.txt'
bestand_testa = '.\data\\voorbeeld12a.txt'
bestand_testb = '.\data\\voorbeeld12b.txt'
bestand_testc = '.\data\\voorbeeld12c.txt'

with open(bestand, 'r') as file:
    grid = [[el for el in x]  for x in [line.strip() for line in file.readlines()]]
    
with open(bestand_testa, 'r') as file:
    test_grida = [[el for el in x]  for x in [line.strip() for line in file.readlines()]]

with open(bestand_testb, 'r') as file:
    test_gridb = [[el for el in x]  for x in [line.strip() for line in file.readlines()]]

with open(bestand_testc, 'r') as file:
    test_gridc = [[el for el in x]  for x in [line.strip() for line in file.readlines()]]

## Code

In [3]:
def create_graph(grid):
    graph = defaultdict(list) 
    n = len(grid)
    edge_numbers = np.array(range(n*n)).reshape(n,n)
    letters = defaultdict(list) 

    for row,col in product(range(n),range(n)):
        edge_number = int(edge_numbers[row][col])
        letters[grid[row][col]].append(edge_number)

        for i in {-1, 1}:
            if row+i in range(n):
                if grid[row+i][col] == grid[row][col]:
                    to_edge = int(edge_numbers[row+i][col])
                    graph[edge_number].append(to_edge)
            if col+i in range(n):
                if grid[row][col+i] == grid[row][col] :
                    to_edge = int(edge_numbers[row][col+i])
                    graph[edge_number].append(to_edge)
    return graph, edge_numbers, n, letters


In [4]:
def find_area(graph, edge_number, visited):
    area = (graph[edge_number]).copy()
    area += [edge_number]
    visited += [edge_number]
    for edge in area:
        if edge in visited:
            pass
        else:
            visited += [edge]
            area += find_area(graph, edge, visited)
    area = list(dict.fromkeys(area))
    return area
    
graph, edge_numbers, n, letters = create_graph(test_grida)
assert find_area(graph, 0, []).sort() == [0,1,2,3,4, 5, 7, 9, 10, 11, 12, 13, 14, 15, 17, 19, 20, 21, 22, 23, 24].sort()
assert find_area(graph, 24, []).sort() == [0,1,2,3,4, 5, 7, 9, 10, 11, 12, 13, 14, 15, 17, 19, 20, 21, 22, 23, 24].sort()
graph, edge_numbers, n, letters = create_graph(test_gridb)
assert find_area(graph, 0, []).sort() ==[0,1,2,3].sort()
assert find_area(graph, 2, []).sort() ==[0,1,2,3].sort()
assert find_area(graph, 7, []) == [7]

In [5]:
def calculate_perimeter(graph, area):
    perimeter = 0
    for edge in area:
        perimeter += (4 - len(graph[edge]))
    return perimeter

graph, edge_numbers, n, letters = create_graph(test_grida)
area = [0,1,2,3,4, 5, 7, 9, 10, 11, 12, 13, 14, 15, 17, 19, 20, 21, 22, 23, 24]
assert calculate_perimeter(graph, area) == 36
graph, edge_numbers, n, letters = create_graph(test_gridb)
assert calculate_perimeter(graph, [0,1,2,3]) == 10
assert calculate_perimeter(graph, [4,5,8,9]) == 8

In [6]:
def fillna_downbet(df, value):
    df = df.copy()
    for col in df:
        non_nans = df[col][~df[col].apply(np.isnan)]
        start, end = non_nans.index[0], non_nans.index[-1]
        df[col].loc[start:end] = df[col].loc[start:end].fillna(value = value)
    return df
            

In [7]:
def find_first_ones(df):
    first_ones = {}
    for col in df:
        first_ones[col] = df[df[col] == 1][col].index[0]
    return first_ones
        

In [45]:
def calculate_perimeter_part2(area,n):
    area_ones = np.empty((n,n))
    area_ones[:] = np.nan
    
    for edge in area:
        row = int(edge/n)
        col = edge%n
        area_ones[row][col] = 1
    
    df = (
        pd.DataFrame(area_ones, dtype=pd.Int64Dtype())
        .dropna(how='all', axis=1)
        .dropna(how='all', axis=0)
        .fillna(0)
        .reset_index(drop=True)
    )
    df_up = df.copy()
    df_down = df.copy()
    df_right = df.copy()
    df_left = df.copy()

    max_row = len(df) - 1
    max_col = len(df.columns) - 1
 
    for index_row, row in df.iterrows():
        for index_col in range(len(df.columns)):
            if index_row == 0:
                df_up.iloc[index_row, index_col] = df.iloc[index_row, index_col] 
            elif df.iloc[index_row-1, index_col] == 1:
                df_up.iloc[index_row, index_col] = 0 

            if index_row == max_row:
                df_down.iloc[index_row, index_col] = df.iloc[index_row, index_col]
            elif df.iloc[index_row+1, index_col] == 1:
                df_down.iloc[index_row, index_col] = 0 

            if index_col == max_col:
                df_right.iloc[index_row, index_col] = df.iloc[index_row, index_col]
            elif df.iloc[index_row, index_col+1] == 1:
                df_right.iloc[index_row, index_col] = 0 

            if index_col == 0:
                df_left.iloc[index_row, index_col] = df.iloc[index_row, index_col]
            elif df.iloc[index_row, index_col-1] == 1:
                df_left.iloc[index_row, index_col] = 0 

    count = 0
    for index_row, row in df.iterrows():
        for index_col in range(len(df.columns)):
            if index_col == 0:
                count += (df_up.iloc[index_row, index_col] + df_down.iloc[index_row, index_col])
            else:
                count += abs(df_up.iloc[index_row, index_col-1] - df_up.iloc[index_row, index_col])
                count += abs(df_down.iloc[index_row, index_col-1] - df_down.iloc[index_row, index_col])

            if index_row == 0:
                count += (df_left.iloc[index_row, index_col] + df_right.iloc[index_row, index_col])
            else:
                count += abs(df_left.iloc[index_row-1, index_col] - df_left.iloc[index_row, index_col])
                # count += abs(df_right.iloc[index_row-1, index_col] - df_right.iloc[index_row, index_col])
                print(index_row, index_col)
                print('index-1', df_right.iloc[index_row-1, index_col])
                print('index', df_right.iloc[index_row, index_col])
                display(df_right)
                print(abs(df_right.iloc[index_row-1, index_col] - df_right.iloc[index_row, index_col]))
                print('--------')
                


    
    
    
    # for index, values in np.ndenumerate(area_ones):
        
    #     print(index)  # operate here

    return  df, df_up, df_down, df_right, df_left, count


df, df_up, df_down, df_right, df_left, count = calculate_perimeter_part2([0,1,2,3,4,5,10,11,12,13,14,15,20, 21, 22, 24, 23], 5) 
display(df)
# display(df_up)
# display(df_down)
display(df_right)
# display(df_left)
print(count)
# assert calculate_perimeter_part2([0,1,4, 8, 12, 13, 14],4) == 8
# assert calculate_perimeter_part2([6, 10, 11, 15], 4) == 8
# assert calculate_perimeter_part2([0,1,2,3,4,5,10,11,12,13,14,15,20, 21, 22, 24, 23], 5) == 12
# area = [0,1,2,3,4, 5, 7, 9, 10, 11, 12, 13, 14, 15, 17, 19, 20, 21, 22, 23, 24]
# assert calculate_perimeter_part2(area, 5) == 20
# assert calculate_perimeter_part2([4,5, 8,9], 4) == 4

1 0
index-1 0
index 1


,0,1,2,3,4
0,0,0,0,0,1
1,1,0,0,0,0
2,0,0,0,0,1
3,1,0,0,0,0
4,0,0,0,0,1


1
--------
1 1
index-1 0
index 0


,0,1,2,3,4
0,0,0,0,0,1
1,1,0,0,0,0
2,0,0,0,0,1
3,1,0,0,0,0
4,0,0,0,0,1


0
--------
1 2
index-1 0
index 0


...

,0,1,2,3,4
0,0,0,0,0,1
1,1,0,0,0,0
2,0,0,0,0,1
3,1,0,0,0,0
4,0,0,0,0,1


1
--------


,0,1,2,3,4
0,1,1,1,1,1
1,1,0,0,0,0
2,1,1,1,1,1
3,1,0,0,0,0
4,1,1,1,1,1


,0,1,2,3,4
0,0,0,0,0,1
1,1,0,0,0,0
2,0,0,0,0,1
3,1,0,0,0,0
4,0,0,0,0,1


8


In [9]:

df

,2,3
0,0,0
1,1,1
2,0,1


In [10]:
max(df.columns)

3

In [11]:
[~(a == 0).all(axis=0)]

NameError: name 'a' is not defined

In [ ]:
def calculate_perimeter_part2(area,n):
    area_ones = np.empty((n,n))
    area_ones[:] = np.nan
    for edge in area:
        area_ones[int(edge/n)] [edge%n] = 1
    df = (
        pd.DataFrame(area_ones, dtype=pd.Int64Dtype())
        .dropna(how='all', axis=1)
        .dropna(how='all', axis=0)
    )

    first_ones = find_first_ones(df)
    df = fillna_downbet(df, 0)
    sum_area_1 = df.sum(axis=0)
    sum_area_0 = (df == 0).sum(axis=0)

    count = 4

    for i, v in sum_area_1.items():
        try:
            if v == sum_area_1[i+1]:
                if first_ones[i] == first_ones[i+1]:
                    pass
                else:
                    count += 4
            else:
                if
                count += 2*sum_area_0[i+1]#(2*sum_area_0[i] + 2*sum_area_0[i+1])
        except:
            break
    count += 2*sum_area_0[i]
    
    return count, df, sum_area_1, sum_area_0
       

area = [0,1,2,3,4, 5, 7, 9, 10, 11, 12, 13, 14, 15, 17, 19, 20, 21, 22, 23, 24]
calculate_perimeter_part2(area, 5)
# assert calculate_perimeter_part2([0,1,4, 8, 12, 13, 14],4) == 8
# assert calculate_perimeter_part2([6, 10, 11, 15], 4) == 8
# assert calculate_perimeter_part2([0,1,2,3,4,5,10,11,12,13,14,15,20, 21, 22, 23, 24], 5) == 12

# assert calculate_perimeter_part2(area, 5) == 20


(12,
    0  1  2  3  4
 0  1  1  1  1  1
 1  1  0  1  0  1
 2  1  1  1  1  1
 3  1  0  1  0  1
 4  1  1  1  1  1,
 0    5
 1    3
 2    5
 3    3
 4    5
 dtype: int64,
 0    0
 1    2
 2    0
 3    2
 4    0
 dtype: int32)

In [ ]:
test_grida

[['O', 'O', 'O', 'O', 'O'],
 ['O', 'X', 'O', 'X', 'O'],
 ['O', 'O', 'O', 'O', 'O'],
 ['O', 'X', 'O', 'X', 'O'],
 ['O', 'O', 'O', 'O', 'O']]

In [ ]:
def calculate_total_price(grid, part = 1):
    graph,edge_numbers, n, letters = create_graph(grid)
    visited =[]
    total_price = 0
    for letter in letters:
        for edge_number in letters[letter]:
            if edge_number in visited:
                pass
            else:
                area = find_area(graph, edge_number, visited)
                visited += area
                if part == 1:
                    perimeter = calculate_perimeter(graph, area)
                else:
                    perimeter = calculate_perimeter_part2(area, n)
                total_price += len(area) * perimeter
    return total_price

calculate_total_price(test_grida, part=2) 
# assert calculate_total_price(test_grida) == 772
# assert calculate_total_price(test_gridb) == 140
# assert calculate_total_price(test_gridc) == 1930
# assert calculate_total_price(test_grida, part=2) == 436
# assert calculate_total_price(test_gridb, part=2) == 80




268

In [ ]:
graph,edge_numbers, n, letters = create_graph(grid)
letters.keys()

dict_keys(['Y', 'R', 'Z', 'I', 'P', 'G', 'K', 'F', 'S', 'J', 'O', 'C', 'N', 'D', 'U', 'E', 'V', 'W', 'X', 'M', 'Q', 'B', 'T', 'L', 'H', 'A'])

In [ ]:
calculate_total_price(grid)

Y
R
Z
I
P
G
K
F
S
J
O
C
N
D
U
E
V
W
X
M
Q
B
T
L
H
A


1374934